In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Mariam\\Desktop\\DataScience\\Deep learning\\ChickenDisease\\Chicken-Disease-Classification-DeeLearning-and-MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Mariam\\Desktop\\DataScience\\Deep learning\\ChickenDisease\\Chicken-Disease-Classification-DeeLearning-and-MLOps'

## Update Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    #returns types and get par from yaml 
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Config Manager

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config= self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config= DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config
    

## Componnents

In [17]:
import  os

import urllib.request as request 
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers= request.urlretrieve(
            url= self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"Downloading {filename} with following info: \n{headers}")
        else:
            logger.info(f"file {get_size(Path(self.config.local_data_file))} already exists")
    def extract_zip_file(self):
        # Exctract zip file into data directory
        # function returns None
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

## Pipeline

In [19]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-03 17:39:17,900: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-03 17:39:17,904: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 17:39:17,909: INFO: common: created directory at: artifacts]
[2024-03-03 17:39:17,912: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-03 17:39:29,414: INFO: 4284437560: Downloading artifacts/data_ingestion/data.zip with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 179C:3CA632:9DBAD7:A577EC:65E47CE7
Accept-Ranges: bytes
Date: Sun, 03 Mar 2024 14:39:19 GMT
Via: 1.1 varnish
X-Served-By: cache-vie6329-VIE
X-Cache: